In [1]:
import os, sys
import scanpy as sc
import numpy as np
import pandas as pd
from importlib import reload
import anndata as ad
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning, ValueWarning, HessianInversionWarning
# warnings.simplefilter('ignore', [ConvergenceWarning, ValueWarning, HessianInversionWarning])
import warnings
warnings.filterwarnings('ignore')

main_dir = os.path.dirname(os.getcwd())
data_dir = f"{main_dir}/data/HoxB8"
os.listdir(data_dir)

['GSE146128_TFnet_exon_counts.csv',
 'GSE146128_exp1_exon_counts.csv',
 'GSE146128_Tfnet_interactions_exon_counts.csv',
 'GSE146128_TFnet_exon_counts.h5ad',
 'GSE146128_Tfnet_interactions_exon_counts.h5ad',
 'GSE146128_exp1_exon_counts.h5ad',
 'TFnet_test_glm.h5ad',
 'GLM_result.pkl',
 'TFnet_test_glm_10xmatrices',
 'TFnext_exon_2khvg_scTransform.rds']

In [2]:
adata_hvg = sc.read_h5ad(f"{data_dir}/TFnet_test_glm.h5ad")
Betas = adata_hvg.varm['GLM_Betas']

In [3]:
adata_hvg

AnnData object with n_obs × n_vars = 1148 × 2000
    obs: 'Plate', 'Well', 'TF', 'Guide'
    var: 'Gene'
    uns: 'NegBin_GLM'
    varm: 'GLM_Betas'
    layers: 'Counts'

In [20]:
adata_hvg.obs['TF']

Sample
RBG18345    Gata3
RBG18346    Gata3
RBG18347    Gata3
RBG18348    Gata3
RBG18349    Gata3
            ...  
RBG24132     Cbfb
RBG24133     Cbfb
RBG24134     Cbfb
RBG24135     Cbfb
RBG24136     Cbfb
Name: TF, Length: 1148, dtype: category
Categories (41, object): ['Cbfa2t3', 'Cbfb', 'Cebpa', 'Cebpb', ..., 'Zbtb17', 'Zfpm1', 'emptyV', 'noB']

In [22]:
adata_hvg.obs['Plate']

Sample
RBG18345     Plate5
RBG18346     Plate5
RBG18347     Plate5
RBG18348     Plate5
RBG18349     Plate5
             ...   
RBG24132    Plate18
RBG24133    Plate18
RBG24134    Plate18
RBG24135    Plate18
RBG24136    Plate18
Name: Plate, Length: 1148, dtype: category
Categories (13, object): ['Plate3', 'Plate4', 'Plate5', 'Plate6', ..., 'Plate15', 'Plate16', 'Plate17', 'Plate18']

# set up api

In [4]:
from MiniPert import model

In [5]:
PGLM = model.Perturb_NBGLM(adata=adata_hvg, perturb_key='TF', plate_key="Plate")

Fitting Negative Binomial regression 
	for 2000 genes 
	with 54 independent variables
Gene ~ Gata3 + Gata2 + Nfe2 + emptyV + R26 +  ...  + Plate13 + Plate15 + Plate16 + Plate17 + Plate18


In [6]:
# this step takes a long time
PGLM.fit(n_jobs=40)

# save the fitted result
PGLM.save_to("/home/wergillius/Project/Alison_Project/result/checkpoint/Hoxb8_2k_fit.ckpt")

start fittings...


100%|██████████| 2000/2000 [11:44<00:00,  2.84it/s] 


 Done.
Adding fitted coeffcients to adata.varm with key `Betas`


In [8]:
len(PGLM.independent_variables)

54

In [33]:
example_result = PGLM.result_dict['ENSMUSG00000000915']

In [47]:
hypotheses = '(Gata2 = 0)'
example_result.f_test(hypotheses)

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=0.23197901301269633, p=0.630156527936335, df_denom=1.1e+03, df_num=1>

# saving and loading

In [ ]:
# initiate a model without fitting
PGLM = model.Perturb_NBGLM(adata=adata_hvg, perturb_key='TF', plate_key="Plate")

# load the saved result
PGLM.load_from("/home/wergillius/Project/Alison_Project/result/checkpoint/Hoxb8_2k_fit.ckpt")

In [7]:
PGLM.model_dict['ENSMUSG00000000555']

In [9]:
PGLM.result_dict['ENSMUSG00000000555'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:     ENSMUSG00000000555   No. Observations:                 1148
Model:                            GLM   Df Residuals:                     1095
Model Family:        NegativeBinomial   Df Model:                           52
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -6398.6
Date:                Mon, 22 May 2023   Deviance:                       70.384
Time:                        13:09:56   Pearson chi2:                     67.0
No. Iterations:                   100   Pseudo R-squ. (CS):             0.1534
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0321      0.030     -1.055      0.292      -0.092       0.028
Gata3         -0.0686      0.318     -0.216      0.829      -0.693       0.555
Gata2          0.0091      0.318      0.029      0.977      -0.615       0.633
Nfe2          -0.0103      0.318     -0.032      0.974      -0.634       0.614
emptyV         0.0600      0.095      0.629      0.529      -0.127       0.247
R26            0.0090      0.102      0.088      0.930      -0.191       0.209
Tcf3           0.0579      0.315      0.184      0.854      -0.559       0.674
Gfi1b         -0.0326      0.315     -0.104      0.917      -0.649       0.584
Egr1          -0.0072      0.315     -0.023      0.982      -0.624       0.609
Cebpa         -0.0618      0.315     -0.197      0.844      -0.678       0.555
Ldb1           0.0750      0.314      0.238      0.812      -0.541       0.691
Lmo2          -0.0288      0.315     -0.092      0.927      -0.646       0.588
Myb           -0.0940      0.318     -0.295      0.768      -0.718       0.530
Rad21         -0.0011      0.318     -0.003      0.997      -0.625       0.623
Myc            0.1085      0.320      0.339      0.734      -0.518       0.735
Cbfa2t3        0.1590      0.315      0.505      0.613      -0.458       0.776
Tal1           0.2202      0.315      0.700      0.484      -0.397       0.837
E2F4           0.1133      0.315      0.360      0.719      -0.504       0.730
Spi1           0.0777      0.314      0.247      0.805      -0.538       0.694
Elf1          -0.0715      0.314     -0.227      0.820      -0.688       0.545
Erg           -0.1521      0.314     -0.484      0.629      -0.768       0.464
Mitf           0.0442      0.317      0.139      0.889      -0.578       0.666
Zfpm1          0.0477      0.315      0.152      0.879      -0.569       0.664
Hoxa9         -0.0245      0.317     -0.077      0.939      -0.647       0.598
Jun           -0.0306      0.314     -0.097      0.922      -0.647       0.586
Fos           -0.0961      0.315     -0.305      0.760      -0.713       0.521
Max           -0.0669      0.315     -0.212      0.832      -0.684       0.550
Meis1         -0.0059      0.322     -0.018      0.985      -0.638       0.626
Runx1          0.0889      0.322      0.276      0.783      -0.543       0.721
Gfi1           0.0487      0.310      0.157      0.875      -0.559       0.656
Ikzf1          0.2621      0.310      0.846      0.397      -0.345       0.869
Nfic          -0.0095      0.310     -0.031      0.976      -0.617       0.598
noB           -0.0498      0.419     -0.119      0.905      -0.872       0.772
Cebpb          0.0792      0.293      0.270      0.787      -0.495       0.653
Ebf1           0.0651      0.290      0.225      0.822      -0.503       0.633
Crtc1         -0.0318      0.293     -0.109      0.914      -0.606       0.542
Kl

In [14]:
DEGs = PGLM.Differerntial_analysis(threshold=0.01)

In [19]:
DEGs['Gata3']
# the name of deg and the p-value

[Index(['ENSMUSG00000000901', 'ENSMUSG00000003882', 'ENSMUSG00000004113',
        'ENSMUSG00000009628', 'ENSMUSG00000012428', 'ENSMUSG00000021190',
        'ENSMUSG00000022037', 'ENSMUSG00000022150', 'ENSMUSG00000022747',
        'ENSMUSG00000026768', 'ENSMUSG00000027750', 'ENSMUSG00000034488',
        'ENSMUSG00000034570', 'ENSMUSG00000035131', 'ENSMUSG00000036172',
        'ENSMUSG00000043421', 'ENSMUSG00000043505', 'ENSMUSG00000043931',
        'ENSMUSG00000047867', 'ENSMUSG00000049134', 'ENSMUSG00000054435',
        'ENSMUSG00000061086', 'ENSMUSG00000068417', 'ENSMUSG00000076617',
        'ENSMUSG00000082596', 'ENSMUSG00000096020', 'ENSMUSG00000096452',
        'ENSMUSG00000103609', 'ENSMUSG00000106497', 'ENSMUSG00000114210'],
       dtype='object', name='Unnamed: 0'),
 array([3.34198713e-04, 1.01689503e-05, 4.59595412e-03, 8.10631242e-03,
        3.12354185e-04, 1.33666019e-09, 5.71015245e-03, 6.61482005e-03,
        5.60865675e-05, 6.90014031e-03, 1.82338813e-06, 7.23732345e-05,


In [17]:
DEGs['Zbtb17']

[Index(['ENSMUSG00000009628', 'ENSMUSG00000034488', 'ENSMUSG00000037922',
        'ENSMUSG00000055675', 'ENSMUSG00000108353'],
       dtype='object', name='Unnamed: 0'),
 array([0.0024663 , 0.0010276 , 0.00017619, 0.00296665, 0.00462703])]